<a href="https://colab.research.google.com/github/dwi11harsh/Digit-Classification-Model/blob/main/Digit_classification_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
# import required libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from google.colab import drive

In [28]:
# mount drive to access MNIST dataset
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
# import training data
data = pd.read_csv('/content/drive/MyDrive/datasets/mnist_train.csv')
data.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
# loading the data
data = np.array(data)
m , n = data.shape
np.random.shuffle(data) #shuffle before splitting into dev and training sets

data_dev = data[0: 1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape

In [31]:
Y_train

array([1, 7, 8, ..., 7, 3, 0])

In [32]:
X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [33]:
X_train[:, 0].shape

(784,)

In [39]:
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
  return np.maximum(0, Z)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def deriv_ReLU(Z):
  return Z > 0

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * deriv_ReLU(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

In [35]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [52]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 500)

Iteration:  0
[9 9 2 ... 9 1 5] [1 7 8 ... 7 3 0]
0.07198305084745762
Iteration:  10
[6 9 6 ... 5 1 5] [1 7 8 ... 7 3 0]
0.24715254237288137
Iteration:  20
[6 9 6 ... 5 3 0] [1 7 8 ... 7 3 0]
0.36971186440677967
Iteration:  30
[1 7 6 ... 7 3 0] [1 7 8 ... 7 3 0]
0.44208474576271184
Iteration:  40
[1 7 2 ... 7 3 0] [1 7 8 ... 7 3 0]
0.4820508474576271
Iteration:  50
[1 7 8 ... 7 3 0] [1 7 8 ... 7 3 0]
0.5086610169491526
Iteration:  60
[1 7 8 ... 7 3 0] [1 7 8 ... 7 3 0]
0.5316271186440678
Iteration:  70
[1 7 8 ... 7 3 0] [1 7 8 ... 7 3 0]
0.5555593220338984
Iteration:  80
[1 7 8 ... 7 3 0] [1 7 8 ... 7 3 0]
0.5770338983050848
Iteration:  90
[1 7 8 ... 9 3 0] [1 7 8 ... 7 3 0]
0.5955932203389831
Iteration:  100
[1 7 8 ... 9 3 0] [1 7 8 ... 7 3 0]
0.6108474576271187
Iteration:  110
[1 7 8 ... 9 3 0] [1 7 8 ... 7 3 0]
0.6252203389830508
Iteration:  120
[1 7 8 ... 9 3 0] [1 7 8 ... 7 3 0]
0.6381016949152543
Iteration:  130
[1 7 8 ... 9 3 0] [1 7 8 ... 7 3 0]
0.651593220338983
Iteration:  14